In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import *
from constants import *
from imports import *
from loaders import *
from train_utils import *
from rollout import RwEvaluator

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

('1.13.1', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'1k5ajf03'

In [3]:
wandb.init(id='7gyoxwyp', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet().to(device) 
m.train()
sum([torch.numel(p) for p in m.parameters()]) / 1000

30484.11

In [5]:
m = try_load_state_dict(m, torch.load(f"{BESPOKE_ROOT}/models/m.torch"))

In [6]:
%%time
from rollout import RwEvaluator
rw_evaluator = RwEvaluator(m, wandb=wandb)

CPU times: user 42 µs, sys: 14 µs, total: 56 µs
Wall time: 63.7 µs


In [7]:
model_stem = "3.10"

In [8]:
%%time

rnn_only = True
rnn_is_first = False

if rnn_only: # m cnn backbone needs to be all loaded up and ready

    if rnn_is_first:
        freeze_model(m, True)
        freeze_model(m.rnn, False)
        m.hidden_init.requires_grad = True
        m.cell_init.requires_grad = True
    else:
        freeze_model(m.backbone, True)
        
    m.backbone.train(False) # freezing bn params
    bs = 16
    bptt = 21 #7 # these have to be multiple of Zloader's internal bptt, currently 3 for bs 32 or 7 for bs 16
    seqlen = bptt*10 #5 #15 
    constant_seq_aug = .0 #TODO can only do constant aug when bptt is one, currently. Fix this.
    n_workers = 5 if seqlen > 100 else 4 if seqlen > 400 else 8 # TODO this doesn't work. dumbass. maxxing out ram here w higher bs. Was 8 w bs 16
    # fewer than five workers we start to have to wait too long for chunk. More is better here.
    dataloader_trn = ZLoader(m, bs=bs, bptt=bptt, seqlen=seqlen, constant_seq_aug=constant_seq_aug, n_workers=n_workers)
else:
    bs = 32
    bptt = 1
    seqlen = 1
    constant_seq_aug = 0 #.9 TODO temp disabled
    n_workers = 8
    dataloader_trn = TrnLoader(bs=bs, bptt=bptt, seqlen=seqlen, constant_seq_aug=constant_seq_aug, n_workers=n_workers)
    
lr = 3e-4
m.use_rnn = rnn_only

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


waiting for chunk


/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been depreca

waiting for chunk
waiting for chunk
waiting for chunk


/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


waiting for chunk
waiting for chunk


/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
waiting for chunk
CPU times: user 4.76 s, sys: 42.7 s, total: 47.5 s
Wall time: 2min 21s


Process Process-2:
Process Process-6:
Process Process-5:
Traceback (most recent call last):
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Process Process-4:
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/beans/bespoke/loaders.py", line 121, in keep_chunk_filled
    time.sleep(0.1)
Traceback (most recent call last):
Process Process-3:
KeyboardInterrupt
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._

In [9]:
opt = torch.optim.Adam(m.parameters(), lr=lr)
#opt = torch.optim.AdamW(m.parameters(), lr=3e-4, weight_decay=.001)

In [10]:
m.backbone.training, m.training

(False, True)

In [11]:
[(n,p.requires_grad) for (n,p) in m.named_parameters()]

[('hidden_init', True),
 ('cell_init', True),
 ('backbone.conv_stem.weight', False),
 ('backbone.bn1.weight', False),
 ('backbone.bn1.bias', False),
 ('backbone.blocks.0.0.conv_dw.weight', False),
 ('backbone.blocks.0.0.bn1.weight', False),
 ('backbone.blocks.0.0.bn1.bias', False),
 ('backbone.blocks.0.0.se.conv_reduce.weight', False),
 ('backbone.blocks.0.0.se.conv_reduce.bias', False),
 ('backbone.blocks.0.0.se.conv_expand.weight', False),
 ('backbone.blocks.0.0.se.conv_expand.bias', False),
 ('backbone.blocks.0.0.conv_pw.weight', False),
 ('backbone.blocks.0.0.bn2.weight', False),
 ('backbone.blocks.0.0.bn2.bias', False),
 ('backbone.blocks.0.1.conv_dw.weight', False),
 ('backbone.blocks.0.1.bn1.weight', False),
 ('backbone.blocks.0.1.bn1.bias', False),
 ('backbone.blocks.0.1.se.conv_reduce.weight', False),
 ('backbone.blocks.0.1.se.conv_reduce.bias', False),
 ('backbone.blocks.0.1.se.conv_expand.weight', False),
 ('backbone.blocks.0.1.se.conv_expand.bias', False),
 ('backbone.block

In [12]:
%%time
"""m.model_stem = model_stem
rw_evaluator.evaluate()"""

CPU times: user 2 µs, sys: 5 µs, total: 7 µs
Wall time: 10 µs


'm.model_stem = model_stem\nrw_evaluator.evaluate()'

In [13]:
for n, mm in m.backbone.named_modules():
    if type(mm) == nn.BatchNorm2d: 
        mm.momentum = .99 #.4

In [14]:
trainer = Trainer(dataloader_trn, m, model_stem=model_stem, opt=opt, wandb=wandb, 
                  rw_evaluator=rw_evaluator, rnn_only=rnn_only)

In [15]:
trainer.reload_state()

Loading model from /home/beans/bespoke/models/m.torch. 
Last modified 3 min ago.
Loading opt from /home/beans/bespoke/models/opt.torch. 
Last modified 3 min ago.
Currently at epoch 10.


In [16]:
copy_cnn_params = False
if copy_cnn_params:
    m.copy_cnn_params_to_rnn() # when first trn rnn, has to be after reload_state.
    print("copied")
    
if rnn_only:
    trainer.loss_manager.update_emas = False
    trainer.loss_manager.loss_emas['te'] = .002 # manual. Bc cnn doesn't cache this.

In [17]:
trainer.loss_manager.update_emas

False

In [18]:
trainer.loss_manager.loss_weights

{'has_stop': 0.1,
 'stop_dist': 0.1,
 'has_lead': 0.1,
 'lead_dist': 0.1,
 'lead_speed': 0.02,
 'lane_width': 0.02,
 'dagger_shift': 0.05,
 'rd_is_lined': 0.02,
 'left_turn': 0.01,
 'right_turn': 0.01,
 'td': 0.03,
 'pitch': 0.1,
 'yaw': 0.1,
 'unc': 0.1,
 'te': 0.05,
 'wp_angles': 1,
 'wp_angles_i': 0.5,
 'wp_curvatures': 0.2,
 'wp_curvatures_i': 0.1,
 'wp_headings': 0.02,
 'wp_headings_i': 0.01,
 'wp_rolls': 0.2,
 'wp_rolls_i': 0.1,
 'wp_zs': 0.02,
 'wp_zs_i': 0.01}

In [19]:
trainer.loss_manager.loss_emas

{'has_stop': 0.005808071112043594,
 'stop_dist': 0.0003114636498602413,
 'has_lead': 0.005251782684192975,
 'lead_dist': 0.0005983034362466439,
 'lead_speed': 0.028805039108986717,
 'lane_width': 0.044827880433625185,
 'dagger_shift': 0.03351293817368379,
 'rd_is_lined': 0.0014623796847056706,
 'left_turn': 0.0009109724650510256,
 'right_turn': 0.0005144749257558216,
 'td': 10.0,
 'pitch': 10.0,
 'yaw': 10.0,
 'unc': 1.3311935594131747,
 'te': 0.002,
 'wp_angles': 0.09521210253818303,
 'wp_angles_i': 0.4515704884783388,
 'wp_curvatures': 0.7389501537666933,
 'wp_curvatures_i': 1.8757729451989167,
 'wp_headings': 0.1309816467120652,
 'wp_headings_i': 0.4639623583839595,
 'wp_rolls': 2.0640401720192587,
 'wp_rolls_i': 0.0022207916021584305,
 'wp_zs': 4.0948565555090894,
 'wp_zs_i': 0.10451367519149185}

In [20]:
trainer.train()

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


736.5 1225.0 151.625 464.75 24.703125 33.65625 40.875 264.25 0.75439453125 210.0
wp_angles_i tensor(0.4006, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings_i tensor(2.7051, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures_i tensor(12.5703, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls_i tensor(0.0032, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs_i tensor(0.2629, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_angles tensor(0.4380, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings tensor(8.5156, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures tensor(12.2969, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls tensor(3.5742, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs tensor(6.4609, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
te tensor(0.0031, device=

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/plotly/matplotlylib/mplexporter/exporter.py:289: MatplotlibDeprecationWarning:


The get_offset_position function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:31: UserWarning:

None of the inputs have requires_grad=True. Gradients will be None



234.0 10944.0 21168.0 5.03515625 621.0 7.8671875 16.171875 957.0 32.6875 13.265625
127.25 49920.0 22544.0 127.9375 5484.0 33.3125 13.109375 291.75 1.068359375 461.75
365.25 857.0 512.5 16.65625 2.88671875 15.0625 43.75 446.0 1.3447265625 437.5
7.44140625 4848.0 3496.0 4.4921875 394.5 1.6025390625 30.609375 155.625 2.08203125 851.0
wp_angles_i tensor(1.2441, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings_i tensor(10.0625, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures_i tensor(16.4219, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls_i tensor(0.0348, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs_i tensor(0.4880, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_angles tensor(0.0262, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings tensor(0.0600, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures tensor(0.1677, device=

58.75 4076.0 4648.0 46.1875 86.4375 42.5625 37.25 198.625 346.0 385.25
wp_angles_i tensor(0.3665, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings_i tensor(8.1250, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures_i tensor(25.8594, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls_i tensor(0.2876, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs_i tensor(0.6787, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_angles tensor(0.0851, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings tensor(0.1494, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures tensor(0.6626, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls tensor(0.7017, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs tensor(4.5703, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
te tensor(0.0086, device='cuda:0', d

...

rd_is_lined tensor(0.0007, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
left_turn tensor(2.5928e-05, device='cuda:0', dtype=torch.float16,
       grad_fn=<MeanBackward0>)
right_turn tensor(6.1154e-05, device='cuda:0', dtype=torch.float16,
       grad_fn=<MeanBackward0>)
unc tensor(1.7891, device='cuda:0', dtype=torch.float16,
       grad_fn=<MseLossBackward0>)
144.125 3712.0 16880.0 34.78125 77.9375 33.65625 67.9375 419.5 1.236328125 46.53125
66.125 3994.0 400.0 13.0625 10.640625 17.703125 23.890625 1250.0 0.2437744140625 348.75
63.8125 188.875 380.0 9.65625 5.22265625 3.40625 45.71875 197.625 0.1363525390625 708.0
143.125 4708.0 9368.0 10.65625 66.0 5.5859375 12.046875 583.0 7.0234375 71.9375


 {'logistical/obs_consumed_per_second': 7.78125, 'logistical/obs_generated_per_second': 24.71875, 'logistical/slowest_runner_obs_per_sec': 2.078125, 'logistical/data_consumption_ratio': 0.31757274, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 2.326

159.5 104.3125 275.5 17.5 6.55078125 1.8525390625 10.71875 288.5 0.220458984375 720.5
39.28125 235.75 2180.0 14.109375 18.125 4.8828125 23.21875 360.5 0.408203125 157.25
1852.0 264.25 24.0625 45.84375 4.046875 5.30859375 13.4921875 74.75 0.79638671875 154.625
loss is nan for wp_headings. Skipping
31.375 893.0 48.0625 20.171875 3.98046875 3.390625 3.056640625 101.6875 0.09954833984375 62.3125
wp_angles_i tensor(0.0370, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings_i tensor(0.5591, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures_i tensor(2.5723, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls_i tensor(0.0005, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs_i tensor(0.0154, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_angles tensor(0.0535, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings tensor(0.1429, device='cuda:0', dtype=torch.float16, g

336.5 617.0 15736.0 148.875 232.0 85.75 19.59375 380.0 1.3505859375 133.75
55.65625 2308.0 37728.0 6.2421875 89.375 3.48828125 11.3828125 377.5 0.343994140625 965.5
wp_angles_i tensor(0.7397, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings_i tensor(11.3750, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures_i tensor(14.2266, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls_i tensor(0.0024, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs_i tensor(0.1088, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_angles tensor(0.0764, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_headings tensor(0.1128, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_curvatures tensor(0.4019, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_rolls tensor(6.1562, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
wp_zs tensor(7.6367, devi

rd_is_lined tensor(0.0002, device='cuda:0', dtype=torch.float16, grad_fn=<MeanBackward0>)
left_turn tensor(3.8028e-05, device='cuda:0', dtype=torch.float16,
       grad_fn=<MeanBackward0>)
right_turn tensor(8.2850e-06, device='cuda:0', dtype=torch.float16,
       grad_fn=<MeanBackward0>)
unc tensor(1.3008, device='cuda:0', dtype=torch.float16,
       grad_fn=<MseLossBackward0>)
321.25 961.5 78.625 28.640625 13.34375 6.32421875 9.6015625 167.875 112.8125 112.1875


 {'logistical/obs_consumed_per_second': 8.09375, 'logistical/obs_generated_per_second': 23.9140625, 'logistical/slowest_runner_obs_per_sec': 2.9140625, 'logistical/data_consumption_ratio': 0.33745561, 'logistical/manual_train_pause': 7.812e-05, 'timing/get batch from dataloader': 2.36689787, 'timing/model forward': 0.00677611, 'timing/calc losses': 0.07866766, 'timing/backwards': 0.17398445, 'timing/get worst': 1.144e-05, 'timing/logging': 0.0503854, 'timing/calc timing': 0.00733673, 'timing/trn update': 2.6840647, 'avg_unc':

57.28125 888.5 7036.0 144.75 21.96875 32.53125 63.53125 368.0 0.366943359375 815.5
9.0546875 3254.0 21568.0 2.67578125 118.0 7.109375 7.5703125 203.875 0.65869140625 132.25
{'logistical/obs_consumed_per_second': 7.59770115, 'logistical/obs_generated_per_second': 23.35632184, 'logistical/slowest_runner_obs_per_sec': 2.0, 'logistical/data_consumption_ratio': 0.32687419, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 2.40827918, 'timing/model forward': 0.00660746, 'timing/calc losses': 0.03448279, 'timing/backwards': 0.18956532, 'timing/get worst': 1.196e-05, 'timing/logging': 0.06212797, 'timing/calc timing': 0.00905957, 'timing/trn update': 2.71014038, 'avg_unc': -2.82405977, 'wp_angles_i': 0.00615679, 'wp_headings_i': 0.0272682, 'wp_curvatures_i': 0.03340426, 'wp_rolls_i': 3.439e-05, 'wp_zs_i': 0.00251644, 'wp_angles': 0.00119003, 'wp_headings': 0.00207493, 'wp_curvatures': 0.00741654, 'wp_rolls': 0.02588761, 'wp_zs': 0.04189816, 'te': 0.00226615, 'has_stop':

In [22]:
t = torch.randn(3,4,5)
t[0,0,0] = torch.inf

In [23]:
t

tensor([[[    inf, -0.7854, -1.3716,  0.8498, -0.0986],
         [ 0.3141, -1.5963, -0.3781, -1.1632,  0.0405],
         [-0.0689, -0.9696,  1.4334,  1.0205,  2.2483],
         [ 1.2481, -0.8255, -0.5951, -0.0256,  0.2744]],

        [[ 0.6463,  1.3897,  0.8417, -0.3971, -1.4846],
         [ 0.3036, -0.4071, -0.0882, -0.6457,  0.2220],
         [ 1.5495,  0.6217, -0.4204,  0.3186,  0.4908],
         [-2.7397, -2.3005,  1.9402, -0.5320, -1.1312]],

        [[ 0.2612,  0.7503, -0.6469,  0.5444, -0.6493],
         [-1.0282, -1.1678, -0.1762,  1.1714,  0.2500],
         [ 0.6518, -0.5325, -1.2205,  0.8746,  0.5583],
         [ 0.3824, -0.7756,  0.8413,  0.3293, -1.3277]]])

In [24]:
t.max()

tensor(inf)

In [20]:
AUX_TARGET_PROPS

['has_stop',
 'stop_dist',
 'has_lead',
 'lead_dist',
 'lead_speed',
 'dagger_shift',
 'lane_width',
 'rd_is_lined',
 'left_turn',
 'right_turn']

In [20]:
trainer.loss_manager.loss_emas

{'has_stop': 0.6581827152218972,
 'stop_dist': 0.1914074304465102,
 'has_lead': 0.6045025000545183,
 'lead_dist': 0.18079276826857715,
 'lead_speed': 0.8834656769419045,
 'lane_width': 4.096574170043749,
 'dagger_shift': 2.512469867257369,
 'rd_is_lined': 0.10581200184235365,
 'left_turn': 0.08856740587054233,
 'right_turn': 0.07401701428385941,
 'td': 10.0,
 'pitch': 10.0,
 'yaw': 10.0,
 'unc': 117.34792707051348,
 'te': 0.002,
 'wp_angles': 0.0859911379087128,
 'wp_angles_i': 4.15318218310992,
 'wp_curvatures': 0.7827851002866839,
 'wp_curvatures_i': 17.866540970943753,
 'wp_headings': 0.12228421566478304,
 'wp_headings_i': 2.901144808827211,
 'wp_rolls': 4.211842307760256,
 'wp_rolls_i': 0.13459417300430104,
 'wp_zs': 5.043518468202303,
 'wp_zs_i': 1.1309414719989206}

In [20]:
m.hidden_init, m.cell_init

(Parameter containing:
 tensor([[[ 0.0015, -0.0488, -0.0292,  ..., -0.0560, -0.0283,  0.0664]]],
        device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([[[-0.3869,  0.0330,  0.0708,  ..., -0.0204,  0.3181, -0.2530]]],
        device='cuda:0', requires_grad=True))

In [21]:
m.training, m.backbone.training 

(True, False)

In [22]:
set_trainer_should_stop(False)

In [28]:
{k:trainer.loss_manager.loss_emas["wp_angles"]/trainer.loss_manager.loss_emas[k] for k,v in trainer.loss_manager.loss_emas.items()}

{'has_stop': 29.734797363340526,
 'stop_dist': 87.87280383313023,
 'has_lead': 40.201898525296386,
 'lead_dist': 127.76564504863305,
 'lead_speed': 0.34846566491395553,
 'lane_width': 3.809900606793288,
 'dagger_shift': 4.134576796863424,
 'rd_is_lined': 16457.381372040072,
 'left_turn': 125.55969474655697,
 'right_turn': 283.74133223646584,
 'td': 0.015300805875045518,
 'pitch': 0.015300805875045518,
 'yaw': 0.015300805875045518,
 'unc': 0.11512757866420895,
 'te': 61.54656219577027,
 'wp_angles': 1.0,
 'wp_angles_i': 0.240031243936724,
 'wp_curvatures': 0.16382346075192508,
 'wp_curvatures_i': 0.003522151816451157,
 'wp_headings': 0.5183519273228973,
 'wp_headings_i': 0.005565440760627469,
 'wp_rolls': 0.027721036050055964,
 'wp_rolls_i': 8.663331955712781,
 'wp_zs': 0.00020285505128895626,
 'wp_zs_i': 0.0035411505624853203}

In [23]:
{'has_stop': 0.005145757574225003,
 'stop_dist': 0.001741244754645775,
 'has_lead': 0.0038059908701619494,
 'lead_dist': 0.0011975680840668381,
 'lead_speed': 0.4390907746627964,
 'lane_width': 0.04016064316156499,
 'dagger_shift': 0.0370069456362572,
 'rd_is_lined': 9.297229935401817e-06,
 'left_turn': 0.0012186080816722507,
 'right_turn': 0.0005392519219686348,
 'td': 10.0,
 'pitch': 10.0,
 'yaw': 10.0,
 'unc': 1.329030459302299,
 'te': 0.002486053701322257,
 'wp_angles': 0.15300805875045517,
 'wp_angles_i': 0.6374505928519476,
 'wp_curvatures': 0.9339813604728601,
 'wp_curvatures_i': 1.44164213359285,
 'wp_headings': 0.2951818073498582,
 'wp_headings_i': 1.49253209788985,
 'wp_rolls': 5.519564942456264,
 'wp_rolls_i': 0.017661571729288113,
 'wp_zs': 754.2728553133405,
 'wp_zs_i': 43.208571917673005


import sys
sys.getsizeof(trainer.loss_manager.loss_emas)

1176

In [16]:
torch.save(trainer.model.state_dict(), f"{BESPOKE_ROOT}/models/m{trainer.model_stem}_e{trainer.current_epoch}.torch")

In [16]:
m.training

Error in callback <function _WandbInit._resume_backend at 0x7f62ef023160> (for pre_run_cell):


Exception: The wandb backend process has shutdown

True

Error in callback <function _WandbInit._pause_backend at 0x7f62e3b863a0> (for post_run_cell):


Exception: The wandb backend process has shutdown

In [ ]:
trainer.opt.param_groups[0]['lr'] = 3e-4

In [28]:
snr

0.15067877149872744



 {'logistical/obs_consumed_per_second': 38.2265625, 'logistical/obs_generated_per_second': 85.1328125, 'logistical/slowest_runner_obs_per_sec': 5.3125, 'logistical/data_consumption_ratio': 0.45004667, 'logistical/manual_train_pause': 0.0, 'timing/get batch from dataloader': 0.00400971, 'timing/model forward': 0.19439457, 'timing/calc losses': 0.00539116, 'timing/backwards': 1.11790375, 'timing/get worst': 0.00264816, 'timing/logging': 0.01570235, 'timing/calc timing': 0.01359244, 'timing/trn update': 1.35364565, 'wp_angles': 0.01003864, 'wp_curvatures': 0.04547036, 'wp_headings': 0.01048879, 'wp_rolls': 0.52840137, 'wp_zs': 0.09037566, 'has_stop': 0.01459547, 'stop_dist': 0.00825622, 'has_lead': 0.01966284, 'lead_dist': 0.00662124, 'lead_speed': 16.11909485, 'dagger_shift': 0.14152849, 'lane_width': 0.12975717, 'rd_is_lined': 0.01347083, 'pitch': 4.037e-05, 'yaw': 4.98e-05, 'unc': 1.57641983, 'logistical/max_param': 32.69097137, 'logistical/lr': 0.0006, 'logistical/mins_since_slowest

In [9]:
%%time

rnn_only = True

if rnn_only: # m cnn backbone needs to be all loaded up and ready
    #freeze_model(m, True)
    #unfreeze_part_of_model(m, "rnn")
    freeze_model(m.backbone, True)
    m.backbone.train(False) # freezing bn params
    bs = 16
    bptt = 14 #35 #8 #7
    seqlen = bptt*5 # needs to be mult of 7
    constant_seq_aug = .0 #TODO can only do constant aug when bptt is one, currently. Fix this.
    dataloader_trn = ZLoader(m, bs=bs, bptt=bptt, seqlen=seqlen)
else:
    bs = 32
    bptt = 1
    seqlen = 1
    constant_seq_aug = .9
    dataloader_trn = TrnLoader(bs=bs, bptt=bptt, seqlen=seqlen, constant_seq_aug=constant_seq_aug)

Launching 3 loader workers


/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:316: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated.

Got first chunk
CPU times: user 1.1 s, sys: 5.72 s, total: 6.83 s
Wall time: 45.2 s


Process Process-3:
Process Process-4:
Process Process-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/beans/bespoke/loaders.py", line 133, in make_chunks
    time.sleep(.1)
  File "/home/beans/miniconda3/envs/py38/lib/python3.8/multiprocessing/process.py", line 1

In [25]:
set_should_stop(True)

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
